In [1]:
import json
import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# import regex as re
# text = open('cardlist.htm').read()
# pattern = r'href="https://gametora\.com/umamusume/supports/([^"]*)"'
# matches = re.findall(pattern, text)
# matches[:5]
# import requests
# linkify = lambda val: f'https://gametora.com/_next/data/dtTM6U5rBo8nmQ3k14c6f/umamusume/supports/{val}.json?id={val}'

# for match in tqdm(matches):
#     result = requests.get(linkify(match))
#     json.dump(json.loads(result.content)['pageProps'],open(f'supports/{match}.json','w'))
#     time.sleep(2)

In [3]:
payload = open('skills_files/8280-5b5309b50b01cbf4.js').read()

In [4]:
import json
parsed = []
sub = payload.split('e.exports=')[8][13:]
while len(sub) > 100:
    # print(len(sub))
    segment = ''
    count = 0
    toSkip = -1
    for i,c in enumerate(sub):
        if i == toSkip:
            continue
        if c == '{':
            count += 1
        elif c == '}':
            count -= 1
        if c == '\\':
            toSkip = i+1
            segment += sub[i+1]
        else:
            segment += c
        if count == 0:
            break
    sub = sub[i+2:]
    # print(segment)
    parsed.append(json.loads(segment.strip()))
skills = {val['id']:val['name_en'] if 'name_en' in val else val['enname'] for val in parsed}
json.dump(skills,open('skills.json','w'))

In [5]:
sub = payload.split('JSON.parse(')[3][2:].split('),a')[0]
parsed = []
while len(sub) > 100:
    # print(len(sub))
    segment = ''
    count = 0
    toSkip = -1
    for i,c in enumerate(sub):
        if i == toSkip:
            continue
        if c == '{':
            count += 1
        elif c == '}':
            count -= 1
        if c == '\\':
            toSkip = i+1
            segment += sub[i+1]
        else:
            segment += c
        if count == 0:
            break
    sub = sub[i+2:]
    # print(segment)
    parsed.append(json.loads(segment.strip()))
    
json.dump({val['id']:val['name_en'] if 'name_en' in val else val['name_en_eon'] for val in parsed},open('effects.json','w'))

In [6]:
sub = payload.split('JSON.parse(')[8]
sub = sub[1:-45]
parsed= []
while len(sub) > 100:
    # print(len(sub))
    segment = ''
    count = 0
    toSkip = -1
    for i,c in enumerate(sub):
        if i == toSkip:
            continue
        if c == '[':
            count += 1
        elif c == ']':
            count -= 1
        if c == '\\':
            toSkip = i+1
            segment += sub[i+1]
        else:
            segment += c
        if count == 0:
            break
    sub = sub[i+2:]
    # print(segment)
    parsed.append(json.loads(segment.strip()))
m = {0:'t',1:'v',2:'d',3:'r'}
costumeEvents = {}
for character in parsed[0]:
    id,data = character
    costumeEvents[id] = {}
    # if 'bakushin' not in str(data).lower():
    #     continue
    for val in data:
        jpname,*event = val
        
        if event[-1][0] == 103:

            eventName = event[-1][-1]
        else:
            eventName = jpname
        costumeEvents[id][eventName] = []
        options = [val[-1] for val in event[0]]
        # print(options)
        for i,opt in enumerate(options):
            # print(i+1)
            temp = []
            for stat in opt:
                d = {}
                for i,val in enumerate(stat):
                    if val is None:
                        continue
                    k = m[i]
                    d[k] = val
                temp.append(d)
            costumeEvents[id][eventName].append(temp)
json.dump(costumeEvents,open('costumeEvents.json','w'))                    
        

In [9]:
sub = payload.split('JSON.parse(')[7]
sub = sub[1:-45]
parsed= []
while len(sub) > 100:
    # print(len(sub))
    segment = ''
    count = 0
    toSkip = -1
    for i,c in enumerate(sub):
        if i == toSkip:
            continue
        if c == '[':
            count += 1
        elif c == ']':
            count -= 1
        if c == '\\':
            toSkip = i+1
            segment += sub[i+1]
        else:
            segment += c
        if count == 0:
            break
    sub = sub[i+2:]
    # print(segment)
    parsed.append(json.loads(segment.strip()))

#id - 
traineeEvents = {}
for character in parsed[0]:
    id,special,choice,date,val1,val2 = character
    traineeEvents[id] = {}
    # print(val2)
    for l in [choice,date,val1]:
        for val in l:
            jpname,*event = val
            
            if event[-1][0] == 103:

                eventName = event[-1][-1]
            else:
                eventName = jpname
            
            traineeEvents[id][eventName] = []
            options = [val[-1] for val in event[0]]
            # print(options)
            for i,opt in enumerate(options):
                # print(i+1)
                temp = []
                for stat in opt:
                    d = {}
                    for i,val in enumerate(stat):
                        if val is None:
                            continue
                        k = m[i]
                        d[k] = val
                    temp.append(d)
                traineeEvents[id][eventName].append(temp)
    print(special)
    if len(val2) > 0:
        traineeEvents[id][val2[-1][2]] = [[{'t':'en','v':'+10'},{'t':'pt','v':'+5'}],[{'t':'en','v':'+30'},{'t':'pt','v':'+10'},{'t':'di'},{'t':'en','v':'+30'},{'t':'pt','v':'+10'},{'t':'sp','v':'-5'},{'t':'po','v':'+5'},{'t':'se','d':'4'}]]
    traineeEvents[id]['Dance Lesson'] = [[{'t':val,'v':'+10'}] for val in special[1]]
    traineeEvents[id]["New Year's Resolutions"] = [[val] for val in [{'t':special[0],'v':'+10'},{'t':'en','v':'+20'},{'t':'pt','v':'+20'}]]
    traineeEvents[id]["New Year's Shrine Visit"] = [[val] for val in [{'t':'en','v':'+30'},{'t':'5s','v':'+5'},{'t':'pt','v':'+35'}]]
    traineeEvents[id]["At Summer Camp (Year 2)"] = [[{'t':'po','v':'+10'}],[{'t':'gu','v':'+10'}]]
json.dump(traineeEvents,open('traineeEvents.json','w'))

        

['po', ['in', 'sp'], 'po', '拝啓、『私』へ', ['ふわっふわスイート', '食べすぎないようにな', '（このまま思う存分食べてもらおう）'], '肌触りが命', 'お供はブラック', '夜に逢えたら', '放っておけない！']
['po', ['po', 'in'], 'po', '世界のオールラウンダー', ['トレードさせていただきますっ！', '食べすぎないようにね', '心ゆくまで堪能しよう！'], '厳正なる抽選の結果……', '今そこにある萌え', '仰げば尊死！\u3000幽体離脱！？', 'あと1曲！\u3000あと1曲だけ……！']
['in', ['st', 'sp'], 'in', '他者感情による心理的効果について', ['お手伝いハンバーガー！', '遠慮しておこう', '折角だし頂けば？'], 'コーヒーの罠？', 'お説教は手短に', '人助けから得たもの？', '研究に夢中！']
['po', ['po', 'sp'], 'po', '君の“理想”は', ['努力の味見', 'それでも食べすぎだと思う', 'ランチには困らないな'], '花と散漫', '閃光と憤怒', 'シール魔を追え！', '花に嵐']
['gu', ['in', 'gu'], 'in', 'Andante', ['エアメサイアの挑戦', 'あまり一気に食べ過ぎないほうが……！', 'どんな味だろうね！'], '心遣いは大切に', '絶対に負けられない戦い', 'エアメサイアのバスケット', '母への想い']
['in', ['sp', 'st'], 'in', 'Life Goes On', ['夜中のラーメン', 'カロリー管理も大事、サプリで済まそう', '我慢はよくない、しっかり食べよう'], 'がびがびスパゲッティ', '代わりなどない', 'チートは許さねェ', 'もっといいモンを作りやがれ']
['sp', ['sp', 'in'], 'sp', 'いつかの、引き出しの中から', ['王冠といえば……', 'レースへの影響は、大丈夫？', '絶対に勝ち取ろう'], '見つかっちゃったらしょうがないのです', '覚えてくれるまで', 'スパイマーちゃんの偵察', 'ファンサービスも大事']
['gu', ['in', 'p

In [10]:
open('acu.txt').read()

'Option 1\n\nRandomly either (~30%)\nAll stats +20\nor (~70%)\nMood -2\nAll stats -15\nGet Night Owl status\n\nOption 2\n\nRandomly either (~45%)\nObtain Corner Recovery ○ skill\nObtain Straightaway Recovery skill\nor (~55%)\nEnergy -20\nMood -2\n\nOption 3\n\nRandomly either (~70%)\nMaximum Energy +12\nEnergy +40\nHeal all negative status effects\nor (~30%)\nEnergy -20\nMood -2\nGet Practice Poor status\n\nOption 4\n\nRandomly either (~85%)\nEnergy +20\nMood +1\nGet Charming ○ status\nor (~15%)\nEnergy -10/-20\nMood -1\n(random) Get Practice Poor status\n\nOption 5\n\nEnergy +10'

In [ ]:
costumes = json.loads(open('skills_files/4054-ebea635d59210eb0.js').read().split('JSON.parse(')[1][1:-7].replace('\\',''))
costumes
# costumes = {f"{val['name_en']} ({val['version']})" if 'version' in val else val['name_en'] : (val['char_id'],val['costume'], 'release_en' in val )  for val in costumes}
# json.dump(costumes,open('costumes.json','w'))

In [10]:
import json
f = 'skills_files/4030-85cc43d100db0360.js'
text = open(f).read()
sub = text.split('JSON.parse(')[1][1:-7]

parsed = json.loads(sub.replace('\\',''))
d = {val['char_id']:val['en_name'] if 'en_name' in val else val['jp_name'] for val in parsed}
json.dump(d,open('charIds.json','w'))

In [11]:

arr = np.array(Image.open('test.png').convert('L'))
def getGameArea(arr):
    arr = (~np.all(arr > 200,axis=0))
    threshold = 100
    padded = np.array([False] + list(arr) + [False])
    diffs = np.diff(padded.astype(int))

    # Start (where it goes from False to True) and end (True to False) indices
    starts = np.where(diffs == 1)[0]
    ends = np.where(diffs == -1)[0]

    # Filter by threshold
    lengths = ends - starts
    valid = np.where(lengths >= threshold)[0]

    if len(valid) > 0:
        first_idx = valid[0]
        start_idx = starts[first_idx]
        end_idx = ends[first_idx] - 1
        
    return (start_idx,end_idx)
getGameArea(arr)

(np.int64(148), np.int64(957))

In [44]:
costumes = json.load(open('costumes.json'))
for k,v in costumes.items():
    if 1011 in v:
        print(k,v)

Grass Wonder [1011, 101101, True]
Grass Wonder (fantasy) [1011, 101116, False]
Grass Wonder (new_year) [1011, 101102, False]


In [4]:
import json
d = json.load(open('traineeEvents.json'))
for k,v in d.items():
    if 'Awkward Honesty' in v:
        print(k)

1008


In [2]:
import json
f = 'skills_files/2080-e05eb4f5931ef54c.js'
text = open(f).read()
sub = text.split('JSON.parse(')[3][1:-45]
sub
parsed = json.loads(sub.replace('\\',''))
d = {val['id']:val['name_en'] if 'name_en' in val else val['name_jp'] for val in parsed}
json.dump(d,open('races.json','w'))

In [28]:
f = 'Support Card List _ Uma Musume _ GameTora_files\8280-5b5309b50b01cbf4.js'
text = open(f).read().split('JSON.parse(')[11][1:-45].replace('\\','')
parsed = json.loads(text)
m = ['t','v','d','r']
scenarioEvents = {}
for scenario in parsed:
    print(scenario)
    eventsWithChoices = scenario[1]
    for event in eventsWithChoices:
        name = event[-1][-1] if event[-1][0] == 103 else event[0]
        choices = [val[1] for val in event[1]]
        processed = []
        for choice in choices:
            temp = []
            for result in choice:
                data = {}
                for i in range(len(result)):
                    data[m[i]] = result[i]
                temp.append(data)
            processed.append(temp)
        scenarioEvents[name] = processed
    # break
json.dump(scenarioEvents,open('scenarioEvents.json','w'))

[1, [['愉快ッ！\u3000密着取材！', [['尊敬すべき理事長です', [['st', '+10'], ['bo', '+5', 9003]]], ['暴走することが多いような……', [['gu', '+10'], ['bo', '+5', 9003]]]], [101, '유쾌! 밀착 취재!'], [102, '愉快！貼身採訪！'], [103, 'Exhilarating! What a Scoop!']], ['トレーナー並の知識', [['力強さが課題なんですが……', [['po', '+10'], ['bo', '+5', 9003]]], ['スピードが課題なんですが……', [['sp', '+10'], ['bo', '+5', 9003]]]], [101, '트레이너 수준의 지식'], [102, '不下於訓練員的知識'], [103, "A Trainer's Knowledge"]], ['上々の面構えッ！', [['では、新しい練習用具をいただけますか？', [['en', '-10'], ['po', '+20'], ['gu', '+20'], ['sk', '+1', 200361]]], ['では、にんじんを分けていただけますか？', [['en', '+30'], ['st', '+20'], ['sk', '+1', 200381]]]], [101, '좋은 표정이로다!'], [102, '表情不錯！'], [103, 'Best Foot Forward!']]], [['二人三脚', [['', [['da', None, [2, 11, 1]], ['fans_minimum', None, 50000], ['nl'], ['in', '+20'], ['pt', '+20'], ['sk', '+1', 200441], ['nl'], ['sl', None, 9004], ['in', '+20'], ['pt', '+20'], ['sk', '+3', 200441]]]], [101, '2인3각'], [102, '兩人三腳'], [103, 'A Three-Legged Race']], ['ハッピーな靴選び', [['', [['po', '+10 (+20 if Aoi Kir